In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import requests
from PIL import Image, ImageOps
import cv2
from scipy import cluster
from sklearn.preprocessing import MinMaxScaler
import imagesize
import random
from sklearn.preprocessing import LabelEncoder
from ipywidgets import interact
import time
from io import BytesIO
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import EarlyStopping
from keras import Sequential
from keras import layers, models
from keras.preprocessing.image import ImageDataGenerator
from tqdm.notebook import tqdm
from keras.metrics import Precision, Recall, Accuracy
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from math import ceil
import os

In [ ]:
#!pip install --upgrade pip

In [ ]:
#!git clone -b preproc_package https://github.com/orlandoalexander/Moda-Personal-Assistant.git

In [ ]:
#%cd Moda-Personal-Assistant
#! pip install .

In [ ]:
#path = '/content/Moda-Personal-Assistant/img'

In [ ]:
#prep = Preproc(path, (299,299), 'fabric', 0.2)

In [ ]:
#X_train, X_test, y_train, y_test = prep.run()

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#!unzip '/content/drive/MyDrive/Le Wagon/Moda-Personal-Assistant/Data/DeepFashion_categorical_images_low/img.zip'

In [ ]:
#PATH_IMG = '/content/img/'
#PATH_DATA = '/content/drive/MyDrive/Le Wagon/Moda-Personal-Assistant/Data/DeepFashion_attributes - train'


In [9]:
PATH_IMG = 'Data/DeepFashion_attributes_image/'
PATH_DATA = 'Data/DeepFashion_attributes_data'


# Attributes

# Categories

## Load raw data

In [52]:
cat_names = pd.read_csv(PATH_DATA+'/list_category.txt',sep ='\s+',header=None).reset_index().rename(columns={0:'category',1:'section','index':'cat_num'})


In [53]:
cat_names['cat_num'] = cat_names['cat_num'].apply(lambda x : x+1)

In [54]:
cat_names.head()

,cat_num,category,section
0,1,Anorak,1
1,2,Blazer,1
2,3,Blouse,1
3,4,Bomber,1
4,5,Button-Down,1


In [55]:
cat_tag = pd.read_csv(PATH_DATA+'/tag_cat_simple.txt',sep='\s+',header=None,names=['category']).rename(columns={'category':'cat_num'})



In [56]:
cat_img = pd.read_csv(PATH_DATA+'/img_simple.txt',sep='\s+',header=None,names=['category']).rename(columns={'category':'img'})

In [57]:
cat_num = cat_img.join(cat_tag).drop(columns='img')


In [58]:
cat_num.head()

,cat_num
0,3
1,33
2,41
3,26
4,2


In [59]:
cat = cat_num.merge(cat_names, on='cat_num', how='left').drop(columns='cat_num')



In [60]:
cat

,category,section
0,Blouse,1
1,Skirt,2
2,Dress,3
3,Jeans,2
4,Blazer,1
...,...,...
19995,Tank,1
19996,Sweater,1
19997,Henley,1
19998,Jumpsuit,3


In [61]:
cat['section'] = cat['section'].map({1:'upper',2:'lower',3:'full body'})

In [62]:
cat_names = pd.read_csv(PATH_DATA+'/list_category.txt',sep ='\s+',header=None).reset_index().rename(columns={0:'category',1:'section','index':'cat_num'})



In [63]:
cat_names['cat_num'] = cat_names['cat_num'].apply(lambda x : x+1)

In [64]:
cat_names.head()

,cat_num,category,section
0,1,Anorak,1
1,2,Blazer,1
2,3,Blouse,1
3,4,Bomber,1
4,5,Button-Down,1


In [65]:
cat_tag = pd.read_csv(PATH_DATA+'/tag_cat_simple.txt',sep='\s+',header=None,names=['category']).rename(columns={'category':'cat_num'})



In [66]:
cat_img = pd.read_csv(PATH_DATA+'/img_simple.txt',sep='\s+',header=None,names=['category']).rename(columns={'category':'img'})

In [67]:
cat_num = cat_img.join(cat_tag)


In [68]:
cat_num.head()

,img,cat_num
0,img/Sweet_Crochet_Blouse/img_00000070.jpg,3
1,img/Classic_Pencil_Skirt/img_00000010.jpg,33
2,img/Strapless_Diamond_Print_Dress/img_00000038...,41
3,img/Mid-Rise_-_Acid_Wash_Skinny_Jeans/img_0000...,26
4,img/Zippered_Single-Button_Blazer/img_00000078...,2


In [69]:
cat = cat_num.merge(cat_names, on='cat_num', how='left').drop(columns='cat_num')


In [70]:
cat['img'] = cat['img'].apply(lambda x: x[4:])

In [71]:
cat

,img,category,section
0,Sweet_Crochet_Blouse/img_00000070.jpg,Blouse,1
1,Classic_Pencil_Skirt/img_00000010.jpg,Skirt,2
2,Strapless_Diamond_Print_Dress/img_00000038.jpg,Dress,3
3,Mid-Rise_-_Acid_Wash_Skinny_Jeans/img_00000010...,Jeans,2
4,Zippered_Single-Button_Blazer/img_00000078.jpg,Blazer,1
...,...,...,...
19995,Embroidered_Peasant_Tank/img_00000044.jpg,Tank,1
19996,Tasmanian_Devil_Sweater/img_00000048.jpg,Sweater,1
19997,Lace-Sleeved_Raglan_Henley/img_00000022.jpg,Henley,1
19998,Floral_Mosaic_Print_Jumpsuit/img_00000017.jpg,Jumpsuit,3


In [72]:
cat['section'] = cat['section'].map({1:'upper',2:'lower',3:'full body'})

In [73]:
data_full = cat

In [79]:
data_full

,img,category,section
0,Sweet_Crochet_Blouse/img_00000070.jpg,Blouse,upper
1,Classic_Pencil_Skirt/img_00000010.jpg,Skirt,lower
2,Strapless_Diamond_Print_Dress/img_00000038.jpg,Dress,full body
3,Mid-Rise_-_Acid_Wash_Skinny_Jeans/img_00000010...,Jeans,lower
4,Zippered_Single-Button_Blazer/img_00000078.jpg,Blazer,upper
...,...,...,...
19995,Embroidered_Peasant_Tank/img_00000044.jpg,Tank,upper
19996,Tasmanian_Devil_Sweater/img_00000048.jpg,Sweater,upper
19997,Lace-Sleeved_Raglan_Henley/img_00000022.jpg,Henley,upper
19998,Floral_Mosaic_Print_Jumpsuit/img_00000017.jpg,Jumpsuit,full body


In [88]:
cat_counts = data_full.category.value_counts() # number of images in each section class (list of tuples) in training data set


In [89]:
cat_counts.mean()

444.44444444444446

In [90]:
cat_counts

Dress          5669
Tee            2289
Blouse         1508
Shorts         1160
Tank           1041
Skirt          1007
Cardigan        918
Sweater         800
Top             775
Jacket          673
Blazer          532
Romper          507
Jumpsuit        503
Jeans           502
Leggings        331
Joggers         294
Hoodie          289
Kimono          200
Sweatpants      162
Coat            157
Cutoffs          96
Sweatshorts      83
Henley           66
Jersey           58
Poncho           56
Jeggings         48
Chinos           45
Culottes         41
Parka            40
Trunks           29
Bomber           23
Turtleneck       14
Flannel          13
Anorak           13
Button-Down      10
Kaftan            9
Capris            8
Robe              7
Onesie            7
Jodhpurs          6
Peacoat           4
Sarong            3
Coverup           2
Caftan            1
Halter            1
Name: category, dtype: int64

## Filter attributes

In [ ]:
data_full

In [ ]:
# If item is sleevless and of lower type, set attribute value to 0:
data_full['sleeveless'] = data_full.apply(lambda x: 0 if ((x['sleeveless'])==1) & (x['section']=='lower') else x['sleeveless'], axis =1)


In [ ]:
data_full

In [ ]:
data_full.drop(columns=['no_dress','no_neckline'],inplace=True)

## Crop and augment images

In [ ]:
data_full

In [ ]:
data_full_attr = data_full.iloc[:,3:-28]

In [ ]:
data_full_attr

In [ ]:
attribute_count = data_full_attr.apply(pd.value_counts)

In [ ]:
attribute_count_T = attribute_count.T.reset_index().rename(columns={'index':'attribute'})

In [ ]:
attribute_count_T

In [ ]:
attribute_count_type = attribute_count_T.merge(attr_names, on='attribute')

In [ ]:
attribute_count_type

In [ ]:
attribute_count_type_mean = attribute_count_type.groupby('attribute_type').median().drop(columns=0)

In [ ]:
attribute_count_type_mean

In [ ]:
attribute_count_mean = attribute_count_type[1].median()
attribute_count_mean

In [ ]:
attr_names_list = attribute_count_type['attribute'].values
attr_names_list

In [ ]:
def landmark_crop(index): # NOT USED AT THE MOMENT !
    observation = data_full.iloc[index,:] # get observation from database according to index
    v1,x1,y1,v2,x2,y2,v3,x3,y3,v4,x4,y4=observation[-24:-12] # landmarks belonging to ALL images
    lm_x = [x1,x2,x3,x4]
    lm_y = [y1,y2,y3,y4]
    lm_v = [v1,v2,v3,v4]
    
    section = data_full.iloc[index,2] # get section type for image
    if section == 'upper' or section == 'full body': # if image is an upper or full body, has 6 landmarks
        v5,x5,y5,v6,x6,y6=observation[-12:-6]
        lm_x.extend([x5,x6])
        lm_y.extend([y5,y6])
        lm_v.extend([v5,v6])
    if section == 'full body': # if image is full body has 8 landmarks
        v7,x7,y7,v8,x8,y8=observation[-6:]
        lm_x.extend([x7,x8])
        lm_y.extend([y7,y8])
        lm_v.extend([v7,v8])
    
    lm_x = np.array(lm_x)
    lm_y = np.array(lm_y)
    lm_x = lm_x[lm_x !=0] # keep only non-zero landmarks 
    lm_y = lm_y[lm_y !=0]
    
    pts = (np.array(list(zip(lm_x,lm_y)))).astype('int') # store coordinates defined by landmarks
    
    img_name = observation[0]
    full_path = PATH_IMG+img_name
    
    img = cv2.imread(full_path) # open image using OpenCV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert image to correct format
    
    x,y,w,h = cv2.boundingRect(pts) # create approximate rectangle using landmark points
    cropped = Image.fromarray(img[y:y+h, x:x+w].copy()) # crop image by bounding rectangle

    return cropped
    

In [ ]:
def format_image(index): # format image
    observation = data_full.iloc[index,:] # get observation from database according to index    
    img_name = observation[0]
    full_path = PATH_IMG+img_name

    # LOAD IMAGES:
    img = Image.open(full_path)
  
    # CROP IMAGES:
    box_x1,box_y1,box_x2,box_y2 = (np.array(observation[-28:-24].values)) # box coords
    cropped = img.crop((box_x1, box_y1, box_x2, box_y2))

    # PAD IMAGES:
    cropped_pad = ImageOps.pad(cropped,(256,256),color=(255, 255, 255)) # pad image to (256,256) with white background
    return cropped_pad


In [ ]:
sample_index = np.random.randint(20000)
image = format_image(sample_index)


plt.imshow(image)

In [ ]:
# fig, axs = plt.subplots(24,10,figsize=(30,100))
# for i in range(24):
#     for j in range(10):
#         index = data_full[data_full[attr_names_list[i]]==1].sample(1).index._data[0]
#         img = format_image(index)
#         img = axs[i][j].imshow(img)
#         axs[i][j].title.set_text(attr_names_list[i])

## Model testing 

In [ ]:
def plot_loss_accuracy(history, title=None):
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    
    # --- LOSS --- 
    
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('Model loss')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylim((0,3))
    ax[0].legend(['Train', 'Test'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # --- ACCURACY
    
    ax[1].plot(history.history['accuracy'])
    ax[1].plot(history.history['val_accuracy'])
    ax[1].set_title('Model Accuracy')
    ax[1].set_ylabel('Accuracy')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Test'], loc='best')
    ax[1].set_ylim((0,1))
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
    
    if title:
        fig.suptitle(title)

### Without image augmentation

In [ ]:
def format_image_model(img_name,box_x1,box_y1,box_x2,box_y2): # format image
    full_path = PATH_IMG+img_name

    
    # LOAD IMAGES:
    img = cv2.imread(full_path) # open image using OpenCV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # CROP IMAGES:
    cropped = Image.fromarray(img[box_y1:box_y2, box_x1:box_x2]) # crop image using box
    
    # PAD IMAGES:
    cropped_pad = ImageOps.pad(cropped,(256,256),color=(255,255,255)) # pad image to (256,256) with white background

    cropped_pad_array = np.asarray(cropped_pad)
    
    return cropped_pad_array

#### Fabric

In [ ]:
fabric_data = data_full.iloc[:,np.r_[0,18:24,27:31]] # select all attributes related to fabric

In [ ]:
fabric_X_names = fabric_data.iloc[:,np.r_[0,7,8,9,10]]

In [ ]:
fabric_X_train = np.array([format_image_model(fabric_X_name[0], *fabric_X_name[1:]) for fabric_X_name in tqdm(fabric_X_names.values)])




In [ ]:
fabric_X_train.shape

In [ ]:
fabric_y_train = fabric_data.iloc[:,1:7]

In [ ]:
fabric_y_train.shape

In [ ]:
fabric_X_train, fabric_X_test, fabric_y_train, fabric_y_test = train_test_split(fabric_X_train, fabric_y_train, test_size =0.2,random_state=2)


In [ ]:
fabric_X_train.shape

In [ ]:
fabric_X_train, fabric_X_val, fabric_y_train, fabric_y_val = train_test_split(fabric_X_train, fabric_y_train, test_size =0.2,random_state=2)


#### Baseline model

In [ ]:
def initialize_model():
  model = Sequential()
  model.add(layers.Conv2D(16, (4,4), input_shape=fabric_X_train[0].shape, padding='valid', activation="relu"))
  model.add(layers.MaxPooling2D(pool_size = (2,2)))
  model.add(layers.Conv2D(32, (3,3), padding='valid', activation="relu"))
  model.add(layers.MaxPooling2D(pool_size = (2,2)))
  model.add(layers.Conv2D(64, (2,2), input_shape=fabric_X_train[0].shape, padding='valid', activation="relu"))
  model.add(layers.MaxPooling2D(pool_size = (2,2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(60,activation='relu'))
  model.add(layers.Dense(6, activation = 'softmax'))
  model.summary()
  return model

In [ ]:
model = initialize_model()
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=[Recall(), Precision(), Accuracy()],
)

In [ ]:
es = EarlyStopping(patience=20,  restore_best_weights=True)

history = model.fit(fabric_X_train, np.array(fabric_y_train), epochs=50, validation_data=(fabric_X_val,fabric_y_val), batch_size=16, callbacks=[es])

In [ ]:
model.evaluate(fabric_X_test,fabric_y_test)

In [ ]:
plot_loss_accuracy(history)


#### VGG16

In [ ]:
# Training image size --> (256,256,3)

In [ ]:
model_vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=fabric_X_train[0].shape) # include top removes classification layer from imagenet dataset
model_vgg16.trainable = False # not trainable weights

## Preprocessing input (normalise data for VGG16 model):
fabric_X_train = preprocess_input(fabric_X_train) 
fabric_X_val = preprocess_input(fabric_X_val)
fabric_X_test = preprocess_input(fabric_X_test)


In [ ]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(60, activation='relu')
dense_layer_2 = layers.Dense(30, activation='relu')
prediction_layer = layers.Dense(6, activation='softmax')


model = models.Sequential([
    model_vgg16,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['recall','precision','accuracy'],
)


es = EarlyStopping(monitor='val_recall', patience=20,  restore_best_weights=True)

history = model.fit(fabric_X_train, fabric_y_train, epochs=50, validation_data=fabric_X_val, batch_size=32, callbacks=[es])

### With image augmentation

In [ ]:
fabric_data_augment = data_full.iloc[:,np.r_[0,18:24,27:31]] # select all attributes related to fabric
fabric_data_augment

In [ ]:
fabric_mean = round(fabric_data_augment.iloc[:,1:7].apply(pd.value_counts).iloc[1,:].mean())
fabric_mean

In [ ]:
fabric_attrs = list(fabric_data_augment.iloc[:,1:7].apply(pd.value_counts).iloc[1,:].items())
fabric_attrs

In [ ]:
def format_image_model(img_name,box_x1,box_y1,box_x2,box_y2): # format image
    full_path = PATH_IMG+img_name
    
    # LOAD IMAGES:
    img = Image.open(full_path)
  
    # CROP IMAGES:
    cropped = img.crop((box_x1, box_y1, box_x2, box_y2))
    
    # PAD IMAGES:
    cropped_pad = ImageOps.pad(cropped,(256,256),color=(255,255,255)) # pad image to (256,256) with white background

    cropped_pad_array = np.asarray(cropped_pad)
    
    return cropped_pad_array

In [ ]:
fabric_data_augment_array = pd.DataFrame()
for  attr, count in fabric_attrs:

  if count >= fabric_mean: # if number of samples is greater than mean number of samples for 'attr'
    print(attr)
    sample_df = fabric_data_augment[fabric_data_augment[attr]==1].sample(fabric_mean,random_state=2)
    sample_df['img'] = sample_df.apply(lambda row: format_image_model(row[0], *row[7:]),axis=1)
    sample_df = sample_df.iloc[:,:7]
  else:
    print(attr)
    sample_values = [0]*count
    remaining_sum = fabric_mean-count
    i = 0
    while remaining_sum != 0:
      sample_values[i]+=1
      remaining_sum-=1
      i = (i+1)%len(sample_values)
    sample_df = pd.DataFrame()
    for index, row in enumerate(fabric_data_augment[fabric_data_augment[attr]==1].values):
      sample = sample_values[index]
      cropped_img_array = format_image_model(row[0], *row[7:])
      temp_oversample_df = pd.concat([pd.DataFrame([row], columns = fabric_data_augment.columns).iloc[:,:7]]*(sample+1),ignore_index=True)
      temp_oversample_df.iloc[0,0] = [cropped_img_array]
      if sample != 0:
        augmented_img_array = augment(cropped_img_array,sample)
        temp_oversample_df.iloc[1:,0] = augmented_img_array
      sample_df = pd.concat([sample_df,temp_oversample_df],axis=0)
  fabric_data_augment_array = pd.concat([fabric_data_augment_array,sample_df],axis=0)

In [ ]:
fabric_data_augment_array

In [ ]:
img_t = fabric_data_augment_array.iloc[37,0]

In [ ]:
plt.imshow(img_t)

In [ ]:
def augment(img_array, samples):
  img_array = img_array.reshape((1,) + img_array.shape)

  # AUGMENT IMAGES:

  # Create an ImageDataGenerator object with the desired transformations
  datagen = ImageDataGenerator(
      horizontal_flip=True,
      width_shift_range=0.2, 
      rotation_range=15,
      fill_mode='constant',
      cval=255
  )

  aug_iter = datagen.flow(img_array, batch_size=1)
  arrays = [aug_iter.next()[0].astype('uint8') for i in range (samples)]
  return arrays

In [ ]:
def display_augmented():
  sample_index = np.random.randint(20000)
  img = format_image(sample_index)
  img_array = np.asarray(img)
  img_array = img_array.reshape((1,) + img_array.shape)

  # AUGMENT IMAGES:

  # Create an ImageDataGenerator object with the desired transformations
  datagen = ImageDataGenerator(
      horizontal_flip=True,
      width_shift_range=0.2, 
      rotation_range=15,
      fill_mode='constant',
      cval=255
  )

  aug_iter = datagen.flow(img_array, batch_size=1)

  # generate samples and plot
  fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(15,15))

  # generate batch of images
  for i in range(5):

    # convert to unsigned integers
    image = next(aug_iter)[0].astype('uint8')
  
    # plot image
    ax[i].imshow(image)
    ax[i].axis('off')

In [ ]:
display_augmented()

In [ ]:
fabric_y_train.apply(pd.value_counts).iloc[1,:]

In [ ]:
def format_image_model(img_name,box_x1,box_y1,box_x2,box_y2): # format image
    full_path = PATH_IMG+img_name

    
    # LOAD IMAGES:
    img = cv2.imread(full_path) # open image using OpenCV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # CROP IMAGES:
    cropped = Image.fromarray(img[box_y1:box_y2, box_x1:box_x2]) # crop image using box
    
    # PAD IMAGES:
    cropped_pad = ImageOps.pad(cropped,(256,256),color=(255,255,255)) # pad image to (256,256) with white background

    cropped_pad_array = np.asarray(cropped_pad)
    
    return cropped_pad_array

#### Fabric

In [ ]:
fabric_data = data_full.iloc[:,np.r_[0,18:24,27:31]] # select all attributes related to fabric

In [ ]:
fabric_X_names = fabric_data.iloc[:,np.r_[0,7,8,9,10]]

In [ ]:
fabric_X_train = np.array([format_image_model(fabric_X_name[0], *fabric_X_name[1:]) for fabric_X_name in tqdm(fabric_X_names.values)])



In [ ]:
fabric_X_train.shape

In [ ]:
fabric_y_train = fabric_data.iloc[:,1:7]

In [ ]:
fabric_y_train.shape

In [ ]:
fabric_X_train, fabric_X_test, fabric_y_train, fabric_y_test = train_test_split(fabric_X_train, fabric_y_train, test_size =0.2,random_state=2)


In [ ]:
fabric_X_train.shape

In [ ]:
fabric_X_train, fabric_X_val, fabric_y_train, fabric_y_val = train_test_split(fabric_X_train, fabric_y_train, test_size =0.2,random_state=2)


##### Baseline model

In [ ]:
def initialize_model():
  model = Sequential()
  model.add(layers.Conv2D(16, (4,4), input_shape=fabric_X_train[0].shape, padding='valid', activation="relu"))
  model.add(layers.MaxPooling2D(pool_size = (2,2)))
  model.add(layers.Conv2D(32, (3,3), padding='valid', activation="relu"))
  model.add(layers.MaxPooling2D(pool_size = (2,2)))
  model.add(layers.Conv2D(64, (2,2), input_shape=fabric_X_train[0].shape, padding='valid', activation="relu"))
  model.add(layers.MaxPooling2D(pool_size = (2,2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(60,activation='relu'))
  model.add(layers.Dense(6, activation = 'softmax'))
  model.summary()
  return model

In [ ]:
model = initialize_model()
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=[Recall(), Precision(), Accuracy()],
)

In [ ]:
es = EarlyStopping(patience=20,  restore_best_weights=True)

history = model.fit(fabric_X_train, np.array(fabric_y_train), epochs=50, validation_data=(fabric_X_val,fabric_y_val), batch_size=16, callbacks=[es])

In [ ]:
model.evaluate(fabric_X_test,fabric_y_test)

In [ ]:
plot_loss_accuracy(history)


##### VGG16

In [ ]:
# Training image size --> (256,256,3)

In [ ]:
model_vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=fabric_X_train[0].shape) # include top removes classification layer from imagenet dataset
model_vgg16.trainable = False # not trainable weights

## Preprocessing input (normalise data for VGG16 model):
fabric_X_train = preprocess_input(fabric_X_train) 
fabric_X_val = preprocess_input(fabric_X_val)
fabric_X_test = preprocess_input(fabric_X_test)


In [ ]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(60, activation='relu')
dense_layer_2 = layers.Dense(30, activation='relu')
prediction_layer = layers.Dense(6, activation='softmax')


model = models.Sequential([
    model_vgg16,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['recall','precision','accuracy'],
)


es = EarlyStopping(monitor='val_recall', patience=20,  restore_best_weights=True)

history = model.fit(fabric_X_train, fabric_y_train, epochs=50, validation_data=fabric_X_val, batch_size=32, callbacks=[es])

# Sections

In [4]:
section = pd.read_csv('Data/DeepFashion_sections_data'+'/fabric.txt',sep=' ',names = ['img','upper_original','lower_original','outer','upper','lower'],header=None,index_col=False).fillna(0)
section['back'] = np.where(section['img'].str.contains('_back'),1,0)
section['full body'] = np.where((section['img'].str.contains('WOMEN-Dresses|WOMEN-Rompers')) & ~section['img'].str.contains('_back'),1,0)
section['outfit'] = np.where((((section['upper_original']!=7) |(section['lower_original']!=7)) & ((section['upper_original']!=7) |(section['outer']!=7))) & (section['full body']==0) & (section['back']==0),1,0)
section = section.drop(columns=['outer','upper_original','lower_original'])

# Landmarks:
landmarks = pd.read_csv('Data/DeepFashion_sections_data'+'/keypoints_loc.txt',sep=' ',usecols=[0,15,16,17,18,29,30,31,32,39,40,41,42],names=['img','x1','y1','x2','y2','x3','y3','x4','y4','x5','y5','x6','y6'], header=None,index_col=False)

# Create full data set:
data_full = section.merge(landmarks, on='img',how='left').dropna()

# Create upper and lower oberservations (to be cropped later):
upper = data_full[data_full['outfit']==1]
lower = data_full[data_full['outfit']==1]
upper['upper'] = upper['upper'].apply(lambda x: 1)
upper['outfit'] = upper['outfit'].apply(lambda x: 0)
lower['lower'] = lower['lower'].apply(lambda x: 1)
lower['outfit'] = lower['outfit'].apply(lambda x: 0)
data_full = pd.concat([data_full,upper,lower],axis=0)
data_full = data_full[(data_full['outfit']==1) | (data_full['full body']==1) | (data_full['back']==1) | (data_full['upper']==1) | (data_full['lower']==1)]

# Drop invalid rows:
data_full = data_full[~data_full.isin([-1]).any(1)] # drop rows with inavlid values

<ipython-input-4-e4ac3a8df5d1>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upper['upper'] = upper['upper'].apply(lambda x: 1)
<ipython-input-4-e4ac3a8df5d1>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upper['outfit'] = upper['outfit'].apply(lambda x: 0)
<ipython-input-4-e4ac3a8df5d1>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

In [5]:
data_full

,img,upper,lower,back,full body,outfit,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,x6,y6
0,MEN-Denim-id_00000080-01_7_additional.jpg,0.0,0.0,0,0,1,267.0,234.0,509.0,229.0,291.0,521.0,478.0,530.0,324.0,1028.0,358.0,1029.0
1,MEN-Denim-id_00000089-01_7_additional.jpg,0.0,0.0,0,0,1,347.0,201.0,598.0,194.0,388.0,519.0,588.0,520.0,492.0,1036.0,511.0,1024.0
2,MEN-Denim-id_00000089-02_7_additional.jpg,0.0,0.0,0,0,1,236.0,234.0,491.0,241.0,266.0,543.0,462.0,534.0,330.0,1037.0,400.0,1034.0
3,MEN-Denim-id_00000089-03_7_additional.jpg,0.0,0.0,0,0,1,225.0,237.0,467.0,211.0,268.0,540.0,470.0,531.0,377.0,1040.0,394.0,1024.0
4,MEN-Denim-id_00000089-04_7_additional.jpg,0.0,0.0,0,0,1,244.0,235.0,480.0,228.0,269.0,545.0,465.0,541.0,341.0,1041.0,399.0,1041.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44074,WOMEN-Tees_Tanks-id_00007969-04_7_additional.jpg,0.0,1.0,0,0,0,221.0,218.0,392.0,180.0,225.0,503.0,416.0,491.0,329.0,999.0,327.0,910.0
44078,WOMEN-Tees_Tanks-id_00007970-01_7_additional.jpg,0.0,1.0,0,0,0,242.0,229.0,446.0,200.0,245.0,444.0,433.0,460.0,292.0,1010.0,345.0,921.0
44082,WOMEN-Tees_Tanks-id_00007976-01_4_full.jpg,0.0,1.0,0,0,0,288.0,225.0,473.0,193.0,297.0,474.0,475.0,486.0,410.0,1022.0,502.0,1026.0
44086,WOMEN-Tees_Tanks-id_00007979-03_4_full.jpg,0.0,1.0,0,0,0,287.0,210.0,466.0,172.0,274.0,467.0,462.0,474.0,359.0,1023.0,361.0,1038.0


In [9]:
for img_name in data_full.iloc[:,0]:
    path = 'Data/test/'
    img = mpimg.imread('Data/DeepFashion_multimodal_images/'+img_name)
    mpimg.imsave(path+img_name, img)